In [1]:
import os
import psycopg2
import glob
import pandas as pd
from Project_Sql_Queries import *

CONNECT TO DATABASE

In [2]:
conn = psycopg2.connect('host=localhost dbname=songstarschema user=postgres password=root port=5432')
cur = conn.cursor()

In [3]:
def get_files(path):
    all_files = []
    for root, dirs, files in os.walk(path):
        files = glob.glob(os.path.join(root, '*.json'))
        for file in files:
            all_files.append(os.path.abspath(file))
    return all_files        

**GET SONG_DATA**

In [4]:
song_files = get_files('data/song_data')

In [5]:
df = pd.read_json(song_files[0], lines=True)
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARJIE2Y1187B994AB7,NaN,NaN,,Line Renaud,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,152.92036,0


# Song

In [6]:
song_data = list(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0])
song_data

['SOSWKAV12AB018FC91', 'Midnight Star', 'ARULZCI1241B9C8611', 0, 335.51628]

In [7]:
cur.execute(insert_song_table, song_data)

In [8]:
conn.commit()

# Artist

In [6]:
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artist_data

['ARULZCI1241B9C8611', 'Luna Orbit Project', '', nan, nan]

In [7]:
cur.execute(insert_artist_table, artist_data)

In [8]:
conn.commit()

In [11]:
cur.close()
conn.close()

# Log data

In [6]:
log_files = get_files('data/log_data/')

In [7]:
df = pd.read_json(log_files[0], lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Theodore,M,0,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,154,None,200,1541290555796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,1541292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
2,None,Logged In,Jahiem,M,0,Miles,NaN,free,"San Antonio-New Braunfels, TX",GET,Home,1.540817e+12,42,None,200,1541299033796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,En Mi Mundo,200,1541300092796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
4,Lifehouse,Logged In,Jahiem,M,2,Miles,203.59791,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,We'll Never Know,200,1541300337796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43


# time

In [8]:
df = df[df['page'] == 'NextSong']
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,1541292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,En Mi Mundo,200,1541300092796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
4,Lifehouse,Logged In,Jahiem,M,2,Miles,203.59791,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,We'll Never Know,200,1541300337796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
5,Olivia Ruiz,Logged In,Jahiem,M,3,Miles,254.74567,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,Cabaret Blanco,200,1541300540796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
7,Jordan Rudess,Logged In,Cecilia,F,1,Owens,1367.84934,free,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.541032e+12,225,Tarkus,200,1541306152796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,Beatsteaks,Logged In,Jacob,M,27,Klein,177.86730,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,72,Jane Became Insane,200,1541365304796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
181,Nouvelle Vague,Logged In,Anabelle,F,0,Simpson,132.70159,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,249,Too Drunk Too F****,200,1541372184796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
182,Mastodon,Logged In,Anabelle,F,1,Simpson,218.87955,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,249,Divinations (Album Version),200,1541372316796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
183,The Killers,Logged In,Anabelle,F,2,Simpson,245.99465,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,249,Sam's Town,200,1541372534796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [9]:
t = pd.to_datetime(df['ts']) # milliseconds -> datetime
t

1     1970-01-01 00:25:41.292603796
3     1970-01-01 00:25:41.300092796
4     1970-01-01 00:25:41.300337796
5     1970-01-01 00:25:41.300540796
7     1970-01-01 00:25:41.306152796
                   ...             
180   1970-01-01 00:25:41.365304796
181   1970-01-01 00:25:41.372184796
182   1970-01-01 00:25:41.372316796
183   1970-01-01 00:25:41.372534796
188   1970-01-01 00:25:41.375113796
Name: ts, Length: 144, dtype: datetime64[ns]

In [10]:
column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_datas = [(time.value, time.hour, time.day, time.week, time.month, time.year, time.weekday()) for time in t]

In [11]:
time_df = pd.DataFrame(data=time_datas, columns=column_labels)
time_df

,timestamp,hour,day,week,month,year,weekday
0,1541292603796,0,1,1,1,1970,3
1,1541300092796,0,1,1,1,1970,3
2,1541300337796,0,1,1,1,1970,3
3,1541300540796,0,1,1,1,1970,3
4,1541306152796,0,1,1,1,1970,3
...,...,...,...,...,...,...,...
139,1541365304796,0,1,1,1,1970,3
140,1541372184796,0,1,1,1,1970,3
141,1541372316796,0,1,1,1,1970,3
142,1541372534796,0,1,1,1,1970,3


In [12]:
for i, row in time_df.iterrows():
    cur.execute(insert_time_table, list(row))
    conn.commit()

# User

In [10]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df

,userId,firstName,lastName,gender,level
1,99,Ann,Banks,F,free
3,43,Jahiem,Miles,M,free
4,43,Jahiem,Miles,M,free
5,43,Jahiem,Miles,M,free
7,6,Cecilia,Owens,F,free
...,...,...,...,...,...
180,73,Jacob,Klein,M,paid
181,69,Anabelle,Simpson,F,free
182,69,Anabelle,Simpson,F,free
183,69,Anabelle,Simpson,F,free


In [11]:
for i, row in user_df.iterrows():
    cur.execute(insert_user_table, list(row))
    conn.commit()

# Songplays

In [14]:
for i, row in df.iterrows():
    cur.execute(select_song, (row.song, row.artist, row.length))
    result = cur.fetchone()
    if result:
        songid, artistid = result
    else:
        songid, artistid = None, None
    songplay_data = (i, row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'],
                     row['location'], row['userAgent'])
    cur.execute(insert_songplays_table, songplay_data)
    conn.commit()